In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import Counter
import math
from math import log

from  pprint import pformat
from collections import namedtuple

In [2]:
def create_data():
    datasets = [['青年', '否', '否', '一般', '否'],
           ['青年', '否', '否', '好', '否'],
           ['青年', '是', '否', '好', '是'],
           ['青年', '是', '是', '一般', '是'],
           ['青年', '否', '否', '一般', '否'],
           ['中年', '否', '否', '一般', '否'],
           ['中年', '否', '否', '好', '否'],
           ['中年', '是', '是', '好', '是'],
           ['中年', '否', '是', '非常好', '是'],
           ['中年', '否', '是', '非常好', '是'],
           ['老年', '否', '是', '非常好', '是'],
           ['老年', '否', '是', '好', '是'],
           ['老年', '是', '否', '好', '是'],
           ['老年', '是', '否', '非常好', '是'],
           ['老年', '否', '否', '一般', '否'],
           ]
    labels = [u'年龄', u'有工作', u'有自己的房子', u'信贷情况', u'类别']
    
    return datasets,labels

In [3]:
datasets,labels=create_data()
train_data=pd.DataFrame(datasets,columns=labels)
train_data2=train_data
train_data2.columns=['Age','Work','Hourse','Loan','Category']

In [4]:
train_data2

,Age,Work,Hourse,Loan,Category
0,青年,否,否,一般,否
1,青年,否,否,好,否
2,青年,是,否,好,是
3,青年,是,是,一般,是
4,青年,否,否,一般,否
5,中年,否,否,一般,否
6,中年,否,否,好,否
7,中年,是,是,好,是
8,中年,否,是,非常好,是
9,中年,否,是,非常好,是


In [5]:
dict(train_data2.Category.value_counts())

{'否': 6, '是': 9}

In [6]:
math.log(8,2)

3.0

In [7]:
def emp_ent(col):
    length=len(col)
    eent=0
    for k,v in dict(col.value_counts()).items():
        p=v/float(length)
        eent+=p*math.log(p,2)
    return -eent
    

In [8]:
emp_ent(train_data2.Category)

0.9709505944546686

In [9]:
def cond_ent(data,feat,label):
    length=len(data[feat])
    cent=0
    for k,v in dict(data[feat].value_counts()).items():
        p=v/float(length)
        cent+=p*emp_ent(data[data[feat]==k][label])
    return cent
    
    

In [10]:
cond_ent(train_data2,'Age','Category')

0.8879430945988998

In [11]:
def gini_func(data,feat,label):
    return emp_ent(data[label])-cond_ent(data,feat,label)

In [12]:
gini_func(train_data2,'Age','Category')

0.08300749985576883

In [13]:
def find_max_gini(data,label):
    col=data.columns.values.tolist()
    col.remove(label)
    mgini=0
    name=None
    for n in col:
        t=gini_func(data,n,label)
        if mgini<t:
            mgini=t
            name=n
    return (name,mgini)
        

In [14]:
find_max_gini(train_data2,'Category')

('Hourse', 0.4199730940219749)

In [15]:
train_data2.Category.unique()[0]

'否'

In [16]:
class tree(namedtuple('dt_tree','feat,value ,label,subtree')):
    def __repr__(self):
        return pformat(tuple(self))

In [17]:
train_data2.Category.value_counts().sort_values().index[-1]

'是'

In [18]:
def dt_train(data,label,epsilon=0.1):
    col=data.columns.values.tolist()
    if len(data[label].unique())==1:
        return tree(None,None,data[label][0],None)
    
    if data.shape[1]==1 and (label !='' or label is not None):
        label=sorted(dict(data[label].value_counts()).items(),key=lambda x:x[1])[-1][0]
        return tree(None,None,label,None)
    
    fname,mgini=find_max_gini(data,label)
    
    if mgini<epsilon:
        return tree(fname,None,data[label].value_counts().sort_values().index[-1],None)
    
    
    
    tl=None
    st=[]
    for v in list(set(data[fname])):
        ll=len(data[data[fname]==v][label].unique())
        if ll ==1:
            tl=data[data[fname]==v][label].unique()[0]
            tv=v
            st.append(tree(None,v,tl,None))    
        else:
#             print(col)
#             print(fname)
            col.remove(fname)
            temp=dt_train(data[data[fname]==v][col],label)
            st.append(tree(None,v,None,temp))
        
    return tree(fname,None,None,st)

In [19]:
print(dt_train(train_data2,'Category'))

('Hourse',
 None,
 None,
 [(None,
 '否',
 None,
 ('Work', None, None, [(None, '否', '否', None), (None, '是', '是', None)])),
  (None, '是', '是', None)])


In [20]:
def dt_pred(tr,last_col,x):
    fv=tr.feat
    if fv is None:
        fv=last_col
    vv=tr.value
    lv=tr.label
    tv=x[fv][0]
    
    if vv==tv and lv is not None:
        return lv
    
    if vv ==tv :
        lv=dt_pred(tr.subtree,fv,x)
    
    if lv is None:
        i=0
        while tr.subtree[i].value != tv:
            i+=1
        lv=dt_pred(tr.subtree[i],fv,x)
    return lv

In [21]:
t_data=np.array(['老年', '否', '否', '一般'])
t_data=t_data.reshape(1,4)
t_col=train_data2.columns.values.tolist()[:-1]
# test_data=pd.DataFrame(np.array(t_data),columns=t_col)
test_data=pd.DataFrame(t_data,columns=t_col)
test_data

,Age,Work,Hourse,Loan
0,老年,否,否,一般


In [22]:
dt=dt_train(train_data2,'Category')
dt_pred(dt,None,test_data)

'否'

Sklearn.tree.DTClassifier

In [23]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
import graphviz
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

In [24]:
def create_data2():
    iris=load_iris()
    df=pd.DataFrame(iris.data,columns=iris.feature_names)
    df['label']=iris.target
    df.columns = ['sepal length', 'sepal width', 'petal length', 'petal width', 'label']
    data = np.array(df.iloc[:100, [0, 1, -1]])
    return data[:,:2],data[:,-1]

In [25]:
X,y=create_data2()
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)

In [26]:
clf=DecisionTreeClassifier()
clf.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [27]:
clf.score(X_test,y_test)

0.9666666666666667

In [28]:
tree_image=export_graphviz(clf,out_file='dtt1.pdf')
with open("dtt1.pdf") as f:
    dot_graph=f.read()

In [29]:
# graphviz.Source(dot_graph)

In [30]:
set(train_data2['Age'])

{'中年', '老年', '青年'}

In [31]:
train_data2['Age'].shape[0]

15

In [32]:
def gini_func2(data,feat,label):
    lv='是'
    length=data.shape[0]
    g={}
    for n in set(data[feat]):
        a=data[data[feat]==n].shape[0] /length
        p1=data[(data[label]==lv) & (data[feat]==n)][label].shape[0] /data[data[feat]==n].shape[0]
        p2=data[(data[label]==lv) & (data[feat]!=n)][label].shape[0] /data[data[feat]!=n].shape[0]
        gv=a * (2*p1*(1-p1))+(1-a)*(2*p2*(1-p2))
        g[n]=gv
#         print('a: '+str(a))
#         print('p: '+str(p))
    return g

In [33]:
gini_func2(train_data2,'Age','Category')

{'中年': 0.48, '老年': 0.44, '青年': 0.44000000000000006}

In [34]:
gini_func2(train_data2,'Hourse','Category')

{'否': 0.26666666666666666, '是': 0.26666666666666666}

In [35]:
gini_func2(train_data2,'Hourse','Category')

{'否': 0.26666666666666666, '是': 0.26666666666666666}

In [36]:
def find_min_gini(data,label):
    col=data.columns.values.tolist()
    col.remove(label)
    name=None
    gm=1
    gn=None
    gn1=None
    for n in col:
        temp=gini_func2(data,n,label)
        
        gn1,gv=sorted(temp.items(),key=lambda x:x[1])[0]
        
        if len(temp)==2 and  len(data[data[n]==gn1][label].unique())!=1:
#             print("test")
            gn1,gv=sorted(temp.items(),key=lambda x:x[1])[-1]
        if gm>gv:
            gm=gv
            gn=gn1
            name=n
    return name,gn,gm

                  

In [37]:
def find_gini(data,label):
    col=data.columns.values.tolist()
    col.remove(label)
    name=None
    gm=1
    gr={}
    for n in col:
        temp=gini_func2(data,n,label)
        tn,tg=sorted(temp.items(),key=lambda x:x[1])[0]
        if len(temp)==2 and  len(data[data[n]==tn][label].unique())!=1:
            tn,tg=sorted(temp.items(),key=lambda x:x[1])[1]
        gr[n]=[tn,tg]
    return gr
        

In [38]:
find_gini(train_data2,'Category')

{'Age': ['老年', 0.44],
 'Hourse': ['是', 0.26666666666666666],
 'Loan': ['一般', 0.32],
 'Work': ['是', 0.32]}

In [39]:
sorted(find_gini(train_data2,'Category').items(),key=lambda x:x[1][1])

[('Hourse', ['是', 0.26666666666666666]),
 ('Work', ['是', 0.32]),
 ('Loan', ['一般', 0.32]),
 ('Age', ['老年', 0.44])]

In [40]:
 gini_func2(train_data2[['Age', 'Loan', 'Category']],'Loan','Category')

{'一般': 0.32, '好': 0.4740740740740741, '非常好': 0.36363636363636365}

In [41]:
find_min_gini(train_data2,'Category')

('Hourse', '是', 0.26666666666666666)

In [42]:
find_min_gini(train_data2[['Age', 'Loan', 'Category']],'Category')

('Loan', '一般', 0.32)

In [43]:
find_min_gini(train_data2[['Age', 'Work', 'Loan', 'Category']],'Category')

('Work', '是', 0.32)

In [44]:
train_data2.columns.values.tolist()

['Age', 'Work', 'Hourse', 'Loan', 'Category']

In [45]:
find_gini(train_data2,'Category')

{'Age': ['老年', 0.44],
 'Hourse': ['是', 0.26666666666666666],
 'Loan': ['一般', 0.32],
 'Work': ['是', 0.32]}

In [46]:
a=find_gini(train_data2,'Category')
print(a)

{'Age': ['老年', 0.44], 'Work': ['是', 0.32], 'Hourse': ['是', 0.26666666666666666], 'Loan': ['一般', 0.32]}


In [48]:
def dt_train2(alldata,data,label,epsilon=0.1):
    col=data.columns.values.tolist()
#     if len(data[label].unique())==1:
#         return tree(None,None,data[label][0],None)
    
    if data.shape[1]==1 :
        return tree(None,None,None,None)
    

    
#     if mgini<epsilon:
#         return tree(fname,None,data[label].value_counts().sort_values().index[-1],None)
    st=[]
    gdict=find_gini(alldata,label)
    m=alldata.shape[1]-data.shape[1]
    fn,v = sorted(gdict.items(),key=lambda x:x[1][1])[m]
    fv=v[0]
    print('fn:'+str(fn))
    print('fv:'+str(fv))
    print('gini:'+str(v[1]))
    print(data)
    lv=sorted(Counter(data[data[fn]==fv][label]).items(),key=lambda x:x[1])[-1][0]
    if data.shape[1]>2:
        lv=sorted(Counter(data[data[fn]==fv][label]).items()
                  ,key=lambda x:x[1])[-1][0]
    else:

        print('lv:'+lv)
        col.remove(fn)
        print(col)
    #     print(data[data[fn]!=fv][col])
        temp=dt_train2(alldata,data[data[fn]!=fv][col],label)
        ts.append(dttree(None,v,None,temp))
    return tree(fn,fv,lv,st)


In [49]:
sorted(Counter(train_data2['Category']).items(),key=lambda x:x[1])

[('否', 6), ('是', 9)]

In [ ]:
print(dt_train2(train_data2,train_data2,'Category'))

In [ ]:
train_data2[(train_data2['Work']!='是') & (train_data2['Hourse']!='是')][['Age','Loan','Category']]

In [ ]:
gini_func2(train_data2[(train_data2['Work']!='是') & (train_data2['Hourse']!='是')]
[['Age','Loan','Category']],'Loan','Category')